## PBL(1): Individual Project Work

1. https://agtechresearch.pythonanywhere.com/ 에 접속하여 회원가입해 주세요. (비밀번호는 단순하게 만드는 것을 권장. 예: 1234)
2. `username` 에 이메일 형식의 아이디를 기입해 주세요.
3. `password` 에 비밀번호를 기입해 주세요.

In [16]:
project = "housingprice"  # 수정하지 마세요
username = "g4312@naver.com"  # 회원가입 시 사용한 이메일아이디 (예시. abc@hello.com)
password = "1234"  # 비밀번호

리더보드 제출을 위한 기본 설정: 아래 코드를 실행해주세요.


In [17]:
import os
import urllib.request

if not os.path.exists("competition.py"):
    url = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/competition/competition.py"
    filename = "competition.py"
    urllib.request.urlretrieve(url, filename)

아래 코드를 실행하여 데이터를 다운로드 받습니다: 3개의 csv 파일이 data 폴더에 다운로드됨

 * dataset.csv: 과거 주택매매 데이터 -> 학습에 사용할 데이터셋
 * problem.csv: 현재 A사가 매매를 고려하고 있는 130건의 주택정보 -> ML 모델에 의하여 예측을 수행하여야 할 데이터셋
 * submission.csv: 리더보드 서버 제출을 위한 파일 형식


In [18]:
import competition

# 파일 다운로드
competition.download_competition_files(project)

100%|██████████| 141k/141k [00:00<00:00, 329kiB/s] 


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# Data 경로 설정
DATA_DIR = "data"

In [20]:
# 학습에 사용할 과거 주택매매 data set 로드 (dataset.csv)
dataset = pd.read_csv(os.path.join(DATA_DIR, "dataset.csv"))

# problem set 로드 (problem.csv)
problemset = pd.read_csv(os.path.join(DATA_DIR, "problem.csv"))

In [21]:
# # 결측치가 있다면 GarageType은 NO Garage로, GarageYrBlt의 값은 0으로 대체
dataset["GarageType"].fillna("No Garage", inplace=True)
dataset["GarageYrBlt"].fillna(0, inplace=True)

dataset.isnull().sum()


Id              0
LotArea         0
Street          0
LotConfig       0
OverallQual     0
OverallCond     0
YearBuilt       0
YearRemodAdd    0
TotalBsmtSF     0
1stFlrSF        0
2ndFlrSF        0
GrLivArea       0
FullBath        0
HalfBath        0
BedroomAbvGr    0
KitchenAbvGr    0
TotRmsAbvGrd    0
Fireplaces      0
GarageType      0
GarageYrBlt     0
GarageCars      0
GarageArea      0
YrSold          0
SalePrice       0
dtype: int64

In [22]:
# #dataset과 problemset의 id, OverallCond, YrSold 지우기
# dataset.drop(["Id", "OverallCond", "YrSold"], axis=1, inplace=True)
# problemset.drop(["Id", "OverallCond", "YrSold"], axis=1, inplace=True)

In [23]:
# dataset을 train set과 val set으로 나누기
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(dataset.drop("SalePrice", axis=1), dataset["SalePrice"], test_size=0.2, random_state=42)

In [24]:
# 범주형 변수들을 모두 one-hot encoding하는데 x_train에 하는걸 그대로 x_val에도 적용
# x_train = pd.get_dummies(x_train)
# x_val = pd.get_dummies(x_val)


In [25]:
# Random over-sampling 적용
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
x_train, y_train = ros.fit_resample(x_train, y_train)



오버샘플링을 하려고 했으나 소수 클래스에 대해 적은 수의 샘플이 있어서 SMOTE 또는 ADASYN이 적절한 이웃을 찾지 못하고 있다
그럼에도 불구하고 오버샘플링은 중요해보인다. 그렇다면 개수가 적은 샘플들은 다른 값으로 치환해 버릴까?

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# 수치형 변수와 범주형 변수를 구분
numeric_features = x_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = x_train.select_dtypes(include=['object']).columns

# 전처리 파이프라인 정의
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Pipeline 정의
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# train 데이터셋에 Pipeline 적용
x_train_preprocessed = pipeline.fit_transform(x_train)

# val 및 test 데이터셋에 Pipeline 적용
x_val_preprocessed = pipeline.transform(x_val)
x_problem_preprocessed = pipeline.transform(problemset)

In [27]:
# lightgbm 모델 학습
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
lgb = lgb.LGBMRegressor()
lgb.fit(x_train_preprocessed, y_train)



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000599 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2134
[LightGBM] [Info] Number of data points in the train set: 8720, number of used features: 32
[LightGBM] [Info] Start training from score 195363.880734


LGBMRegressor()

In [31]:
# lightgbm 모델의 최적의 파라미터 찾기
from sklearn.model_selection import GridSearchCV
param_grid = {
    'num_leaves': [31, 127],
    'reg_alpha': [0.1, 0.5],
    'min_data_in_leaf': [30, 50, 100, 300],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
}
Grid_lgb = GridSearchCV(lgb, param_grid, cv=5)
Grid_lgb.fit(x_train_preprocessed, y_train)
print(Grid_lgb.best_params_)

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000972 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2103
[LightGBM] [Info] Number of data points in the train set: 6976, number of used features: 30
[LightGBM] [Info] Start training from score 209324.929042
[LightGBM] [

In [36]:
print(Grid_lgb.best_params_)


{'lambda_l1': 1.5, 'lambda_l2': 1, 'min_data_in_leaf': 30, 'num_leaves': 127, 'reg_alpha': 0.1}


In [38]:
# best_params_를 이용하여 lightgbm 모델 재학습
lgb_best_params = Grid_lgb.best_params_
lgb_model = lgb.(**lgb_best_params)
lgb_model.fit(x_train_preprocessed, y_train)

# rmse 계산
y_pred = lgb_model.predict(x_val_preprocessed)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(rmse)


AttributeError: 'LGBMRegressor' object has no attribute 'model'

In [28]:
from sklearn.metrics import mean_squared_error

train_pred = lgb.predict(x_train_preprocessed)
test_pred = lgb.predict(x_val_preprocessed)

print("Train RMSE :", mean_squared_error(y_train, train_pred) ** 0.5)
print("Test RMSE :", mean_squared_error(y_val, test_pred) ** 0.5)

Train RMSE : 5421.580459199216
Test RMSE : 30570.304113905506


In [29]:
problem_pred = lgb.predict(x_problem_preprocessed)

In [30]:
# 리더보드 서버 제출을 위한 파일 생성
submission = pd.read_csv(os.path.join(DATA_DIR, "submission.csv"))
submission["SalePrice"] = problem_pred

# 예측 결과 화면에 출력 후 제출
display(submission)
competition.submit(project, username, password, submission)

,Id,SalePrice
0,1341,133682.482793
1,1342,139899.633260
2,1343,168552.109592
3,1344,145280.160102
4,1345,95009.443566
...,...,...
125,1466,299893.608459
126,1467,173711.377833
127,1468,77942.384811
128,1469,100558.335663


아이디:  g4312@naver.com
파일명:  submissions\20240513-204002-submission.csv
[제출에 성공하였습니다]
제출 결과: 25975.06638475992
